In [6]:
import pandas as pd
import cobra

> <span style="color:blueviolet">**Save list of genes in Recon3DModel_301**</span> : 

>             The total number of genes is  2248
>             This list includes different isoformes (e.g. 5224.1 & 5224.2)

In [7]:
pwd


'/Users/stefaniaastrologo/Documents/Rohit_project/tissueModel/Bulk_scRNAseq_Recon3_ids'

In [8]:
recon3   = cobra.io.read_sbml_model('../Recon3D_301_SBML/Recon3DModel_301.xml')
recon3_genes = list(set([(x.name) for x in recon3.genes]))
len(set(recon3_genes))

2248

>             The metadata dataset do not consider isoforms, the number of IDs drops to 1884

In [9]:
recon3_genes = list(set([int(x.name.split('.')[0]) for x in recon3.genes]))
len(set(recon3_genes))

1884

> <span style="color:blueviolet">**Transcriptomics dataset of six diffrent cell lines ('LTED', 'LTED_F', 'LTED_T', 'MCF7', 'MCF7_F', 'MCF7_T')**</span> : 


In [10]:
meta_data = pd.read_table('count_results.txt')
meta_data.columns = ['LTED', 'LTED_F', 'LTED_T', 'MCF7', 'MCF7_F', 'MCF7_T']
len([x for x in recon3_genes if x not in meta_data.index])
meta_data.head()

,LTED,LTED_F,LTED_T,MCF7,MCF7_F,MCF7_T
1,2.108951,2.090415,1.634452,1.555154,0.529875,2.298877
10,0.106588,0.000000,0.000000,0.000000,0.000000,0.000000
100,0.916292,0.858569,0.238684,5.518615,1.310894,0.839278
1000,0.831648,2.764159,0.122624,1.446760,3.311230,2.127147
10000,0.076489,0.044794,0.224150,0.014218,0.153884,0.017515


>             The row names is the entrez_id (same id used by Recon3)
>             I filter only the rows relative to the metabolic genes
>             
>             25 genes are present in Recon3, but not in the dataset due to 
>             old mapping of the transcriptomics
>             
>             Those rows will be filled with zeros 

In [11]:
meta_data = pd.read_table('count_results.txt')
meta_data.columns = ['LTED_FPKM', 'LTED_F_FPKM', 'LTED_T_FPKM', 'MCF7_FPKM', 'MCF7_F_FPKM', 'MCF7_T_FPKM']
meta_data = meta_data.loc[recon3_genes]
meta_data = meta_data.fillna(0)

/Users/stefaniaastrologo/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3 FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


>             I make a new dataframe from each cell lines;
>             each df has 2 columns ['entrez_id', '*_FPKM']
>             the entrez_id is now the 'gene name' used in Recon3 
>             since we have one FPKM value for each isorform ...
>             diffrent isoform for the same ID have

In [12]:
All_recon3_genes = list([x.name for x in recon3.genes])
len(All_recon3_genes)

2248

In [13]:
LTED   = pd.DataFrame(columns=['entrez_id', 'LTED_FPKM'])
LTED_F = pd.DataFrame(columns=['entrez_id', 'LTED_F_FPKM'])
LTED_T = pd.DataFrame(columns=['entrez_id', 'LTED_T_FPKM'])
MCF7   = pd.DataFrame(columns=['entrez_id', 'MCF7_FPKM'])
MCF7_F = pd.DataFrame(columns=['entrez_id', 'MCF7_F_FPKM'])
MCF7_T = pd.DataFrame(columns=['entrez_id', 'MCF7_T_FPKM'])

df_list = [LTED, LTED_F, LTED_T, MCF7, MCF7_F, MCF7_T]

for gene_name in All_recon3_genes:
    
    for Dataframe in df_list:
        
        entrez_id = int(gene_name.split('.')[0])
        value = meta_data.loc[entrez_id][Dataframe.columns[1]]
        
        Dataframe.loc[len(Dataframe)] = [gene_name, value]
# for Dataframe in df_list:
#     Dataframe.to_csv(str(Dataframe.columns[1])+'_Recon3_ids.csv')
